MSSV: 20521935 HovaTen: NguyenThiNgocThao

In [2]:
import findspark

In [4]:
findspark.init()

In [5]:
import pyspark 
from pyspark.sql  import  SparkSession
from pyspark.sql.functions import col,to_date,date_format
from datetime import datetime
import pyspark.sql.functions as f
from pyspark.sql.types import DoubleType

In [6]:
from pyspark.context import SparkContext
spark=SparkSession.builder.getOrCreate()

In [7]:
movies=spark.read.option("header",True).format("csv").load("movies_small.csv")
movies.printSchema()
movies.show(5,truncate=False)

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
+-------+----------------------------------+-------------------------------------------+
only showing top 5 rows



In [8]:
ratings=spark.read.option("header",True).format("csv").load("ratings_small.csv")
ratings.printSchema()
ratings.show(5,truncate=False)

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|1     |1      |4.0   |964982703|
|1     |3      |4.0   |964981247|
|1     |6      |4.0   |964982224|
|1     |47     |5.0   |964983815|
|1     |50     |5.0   |964982931|
+------+-------+------+---------+
only showing top 5 rows



In [9]:
movies=movies.withColumn('title',f.trim(col('title')))

In [10]:
movies=movies.withColumn('title', f.regexp_replace('title', '"', ''))

In [11]:
movies=movies.withColumn('year',movies.title.substr(-5,4))

In [12]:
movies.sort('year').show(5)

+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+----+
| 171631|Maria Bamford: Ol...|  (no genres listed)| Bab|
| 143410|          Hyena Road|  (no genres listed)| Roa|
|  32898|Trip to the Moon,...|Action|Adventure|...|1902|
|  49389|The Great Train R...|       Crime|Western|1903|
| 140541|The Electric Hote...|Animation|Comedy|...|1908|
+-------+--------------------+--------------------+----+
only showing top 5 rows



In [13]:
movies=movies.withColumn('year',col('year').cast('int')).sort('year')

In [14]:
movies.show(5,truncate=False)

+-------+-----------------+----------------------+----+
|movieId|title            |genres                |year|
+-------+-----------------+----------------------+----+
|140956 |Ready Player One |Action|Sci-Fi|Thriller|null|
|149334 |Nocturnal Animals|Drama|Thriller        |null|
|143410 |Hyena Road       |(no genres listed)    |null|
|156605 |Paterson         |(no genres listed)    |null|
|40697  |Babylon 5        |Sci-Fi                |null|
+-------+-----------------+----------------------+----+
only showing top 5 rows



In [15]:
movies.where(movies.year.isNull() ).show(truncate=False)

+-------+---------------------------------------------------+----------------------+----+
|movieId|title                                              |genres                |year|
+-------+---------------------------------------------------+----------------------+----+
|40697  |Babylon 5                                          |Sci-Fi                |null|
|140956 |Ready Player One                                   |Action|Sci-Fi|Thriller|null|
|143410 |Hyena Road                                         |(no genres listed)    |null|
|147250 |The Adventures of Sherlock Holmes and Doctor Watson|(no genres listed)    |null|
|149334 |Nocturnal Animals                                  |Drama|Thriller        |null|
|156605 |Paterson                                           |(no genres listed)    |null|
|162414 |Moonlight                                          |Drama                 |null|
|167570 |The OA                                             |(no genres listed)    |null|
|171495 |C

In [16]:
ratings.withColumn('rating',col('rating').cast(DoubleType())).printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: string (nullable = true)



1. Show the number of movies made in each year. The results are sorted by year.

In [158]:
filterr = movies.filter(col("year").isNotNull())
counter = filterr.groupBy("year").count()
sort = counter.sort("year")
sort.show(100)

+----+-----+
|year|count|
+----+-----+
|1902|    1|
|1903|    1|
|1908|    1|
|1915|    1|
|1916|    4|
|1917|    1|
|1919|    1|
|1920|    2|
|1921|    1|
|1922|    1|
|1923|    4|
|1924|    5|
|1925|    4|
|1926|    5|
|1927|    7|
|1928|    4|
|1929|    4|
|1930|    5|
|1931|   14|
|1932|    9|
|1933|   12|
|1934|   11|
|1935|   13|
|1936|   18|
|1937|   16|
|1938|   15|
|1939|   23|
|1940|   25|
|1941|   18|
|1942|   23|
|1943|   10|
|1944|   16|
|1945|   17|
|1946|   23|
|1947|   20|
|1948|   20|
|1949|   25|
|1950|   21|
|1951|   22|
|1952|   16|
|1953|   30|
|1954|   23|
|1955|   36|
|1956|   30|
|1957|   33|
|1958|   31|
|1959|   37|
|1960|   37|
|1961|   34|
|1962|   40|
|1963|   39|
|1964|   43|
|1965|   47|
|1966|   42|
|1967|   42|
|1968|   42|
|1969|   35|
|1970|   33|
|1971|   47|
|1972|   39|
|1973|   59|
|1974|   45|
|1975|   42|
|1976|   44|
|1977|   63|
|1978|   59|
|1979|   69|
|1980|   89|
|1981|   92|
|1982|   87|
|1983|   83|
|1984|  101|
|1985|  126|
|1986|  139|

2. Show the number of movies belonging to each genre made in each year. The results are sorted by year. The result
of each genre is displayed in each column

In [159]:
from pyspark.sql.functions import explode, split, count

In [160]:
data = filterr.withColumn("genre", explode(split(col("genres"), "\\|")))

In [161]:
count_num = data.groupBy("year").pivot("genre").agg(count("*")).na.fill(0)

In [162]:
result = count_num.orderBy("year")

In [165]:
result.show(100)

+----+------------------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+----+-------+-------+-------+------+--------+---+-------+
|year|(no genres listed)|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|IMAX|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+----+------------------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+----+-------+-------+-------+------+--------+---+-------+
|1902|                 0|     1|        1|        0|       0|     0|    0|          0|    0|      1|        0|     0|   0|      0|      0|      0|     1|       0|  0|      0|
|1903|                 0|     0|        0|        0|       0|     0|    1|          0|    0|      0|        0|     0|   0|      0|      0|      0|     0|       0|  0|      1|
|1908|                 0|     0|        0|        1|       0|     1|    0|          0|    0|      0|        0|     0|   0|   

3.For each userID, show the average rating of that user for each genre. The results are sorted by userID. The result of
each genre is displayed in each column.

In [166]:
joining = ratings.join(data,['movieId'],'inner')
joining.show(5)

+-------+------+------+---------+----------------+--------------------+----+---------+
|movieId|userId|rating|timestamp|           title|              genres|year|    genre|
+-------+------+------+---------+----------------+--------------------+----+---------+
|      1|     1|   4.0|964982703|Toy Story (1995)|Adventure|Animati...|1995|  Fantasy|
|      1|     1|   4.0|964982703|Toy Story (1995)|Adventure|Animati...|1995|   Comedy|
|      1|     1|   4.0|964982703|Toy Story (1995)|Adventure|Animati...|1995| Children|
|      1|     1|   4.0|964982703|Toy Story (1995)|Adventure|Animati...|1995|Animation|
|      1|     1|   4.0|964982703|Toy Story (1995)|Adventure|Animati...|1995|Adventure|
+-------+------+------+---------+----------------+--------------------+----+---------+
only showing top 5 rows



In [186]:
user = joining.groupBy('userId').pivot('genre').agg(f.avg('rating').alias('avg_rating')).fillna(0)

In [187]:
user_sort = user.orderBy('userId')

In [188]:
user_sort.show(100)

+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|userId|(no genres listed)|            Action|         Adventure|         Animation|          Children|            Comedy|             Crime|       Documentary|             Drama|           Fantasy|         Film-Noir|            Horror|              IMAX|           Musical|           Mystery|           Romance|            Sci-Fi|          Thriller|               War|           Western|
+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----

4. For each movie, show the name, the year, the number of ratings, and the average rating (from all users) of each
movie. The results are sorted by the years and then by the names of the movies.

In [245]:
total = movies.join(ratings, on="movieId", how="left").filter(col("year").isNotNull())

In [246]:
grouped_data = total.groupBy("year", "title").agg(
    avg("rating").alias("Average_rating"),
    count("rating").alias("Num_rating")
)

In [247]:
result4= grouped_data.orderBy("year", "title")

In [249]:
result4.show(100)

+----+--------------------+------------------+----------+
|year|               title|    Average_rating|Num_rating|
+----+--------------------+------------------+----------+
|1902|Trip to the Moon,...|               3.5|         5|
|1903|The Great Train R...|               2.5|         2|
|1908|The Electric Hote...|               4.0|         1|
|1915|Birth of a Nation...|               2.0|         1|
|1916|20,000 Leagues Un...|              3.75|         2|
|1916|Intolerance: Love...|               3.5|         1|
|1916|    Rink, The (1916)|               3.5|         1|
|1916|   Snow White (1916)|               3.5|         1|
|1917|Immigrant, The (1...|               4.5|         1|
|1919|Daddy Long Legs (...|               2.0|         1|
|1920|Cabinet of Dr. Ca...| 3.857142857142857|         7|
|1920|     One Week (1920)|               3.5|         1|
|1921|     Kid, The (1921)|               4.1|         5|
|1922|Nosferatu (Nosfer...|           3.53125|        16|
|1923|Hunchbac

5. For each user ID, show the genre that received highest average rating from that user and the list of top 5 movies
belonging to that genre that receive highest average rating from all user and haven’t been rated by that user (For
example, ‘Action’ is the genre that received highest average rating from user ID X. Among ‘Action’ movies that hasn’t
been rated by user ID X, you are supposed to show top 5 movies that received highest average rating from all users).
The results are sorted by the user ID.

In [317]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, desc, row_number, collect_list, concat_ws
from pyspark.sql.window import Window

In [329]:
phims = movies.withColumn( "genres_array", f.split("genres", "\|")).withColumn('last_genres', f.element_at("genres_array", -1))

In [330]:
nguoirating = ratings.join(phims, on='movieid', how='left').groupBy('userId', 'last_genres').agg(f.avg('rating').alias('tbc_rating'))

In [331]:
window_spec = Window.partitionBy('userid').orderBy(col('tbc_rating').desc())

In [332]:
nguoidung = nguoirating.withColumn('rank', f.row_number().over(window_spec)).where(col('rank') == 1).select('userId', 'last_genres')

In [333]:
wsm = Window.partitionBy('last_genres').orderBy(col('tbc_rating').desc())

In [334]:
nguoidungtop = nguoidung.join(phims, on='last_genres', how='left') \
.join(nguoirating, on=['userId', 'last_genres'], how='left') \
.withColumn('rank',f.row_number().over(wsm)) \
.where(col('rank') <= 5) \
.groupBy('userid') \
.agg(f.max('last_genres').alias('Highest_rated_genre_name'), f.collect_list('title').alias('Top_5_unrated_movies_with_highest_rating'))

In [335]:
nguoidungtop = nguoidungtop.orderBy('userId') 

In [336]:
user_top_movies.orderBy('userId').show(truncate=False)

+------+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userid|Highest_rated_genre_name|Top_5_unrated_movies_with_highest_rating                                                                                                                                                                      |
+------+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1     |Mystery                 |[Murder on the Orient Express (2017), Buster's Mal Heart (2017), Kizumonogatari III: Cold Blood (2017), Kizumonogatari II: Passionate Blood (2016), Kizumonogatari Part 1: Tekketsu (2016)]                   |
|106   |Drama                   |[Fl